In [1]:
import os
import warnings

warnings.filterwarnings(action="ignore")

from openff.toolkit.typing.engines.smirnoff import ForceField
from paprika.build.system.taproom import BuildTaproomAPR
from pkg_resources import resource_filename
from tqdm.auto import tqdm

In [2]:
os.environ["INTERCHANGE_EXPERIMENTAL"] = "1"

* The `BuildTaproomAPR` class requires the OpenFF-Interchange modules. Install in your conda environment with:

`conda install -c conda-forge openff-interchange`

## 01) Build System in explicit solvent

In [3]:
# Define force field (OpenFF)
force_field = ForceField("openff-2.0.0.offxml")

In [4]:
system = BuildTaproomAPR(
    host_code="acd",
    guest_code="bam",
    n_water=2200,
    force_field=force_field,
    working_folder="explicit_solvent",
    disable_progress=False,
)

In [5]:
# Build APR files -- parallelize over 4 CPUs
system.build_system(n_cpus=4, clean_files=True)

generating files `pull-s`:   0%|          | 0/46 [00:00<?, ?it/s]

/Users/jsetiadi/anaconda3/envs/paprika-dev/lib/python3.10/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return func(*args, **kwargs)
/Users/jsetiadi/anaconda3/envs/paprika-dev/lib/python3.10/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return func(*args, **kwargs)
/Users/jsetiadi/anaconda3/envs/paprika-dev/lib/python3.10/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this metho

generating files `attach-s`:   0%|          | 0/15 [00:00<?, ?it/s]

generating files `pull-p`:   0%|          | 0/46 [00:00<?, ?it/s]

generating files `attach-p`:   0%|          | 0/15 [00:00<?, ?it/s]

...

applying restraints `attach-s`:   0%|          | 0/15 [00:00<?, ?it/s]

applying restraints `attach-p`:   0%|          | 0/15 [00:00<?, ?it/s]

applying restraints `pull-s`:   0%|          | 0/46 [00:00<?, ?it/s]

applying restraints `pull-p`:   0%|          | 0/46 [00:00<?, ?it/s]

applying restraints `release`:   0%|          | 0/15 [00:00<?, ?it/s]

## 02) Build System in implicit solvent

In [6]:
# Define force field (OpenFF)
gbsa = resource_filename(
    "openff.toolkit",
    os.path.join("data", "test_forcefields", "GBSA_OBC2-1.0.offxml"),
)
force_field = ForceField("openff-2.0.0.offxml", gbsa)

In [7]:
system = BuildTaproomAPR(
    host_code="acd",
    guest_code="bam",
    n_water=None, #<-- Set to None to build system in vacuum
    force_field=force_field,
    working_folder="implicit_solvent",
    disable_progress=False,
)

In [8]:
# Build APR files -- parallelize over 4 CPUs
system.build_system(n_cpus=4, clean_files=True)

generating files `pull-s`:   0%|          | 0/46 [00:00<?, ?it/s]

generating files `attach-s`:   0%|          | 0/15 [00:00<?, ?it/s]

generating files `pull-p`:   0%|          | 0/46 [00:00<?, ?it/s]

generating files `attach-p`:   0%|          | 0/15 [00:00<?, ?it/s]

generating files `release`:   0%|          | 0/15 [00:00<?, ?it/s]

applying restraints `attach-s`:   0%|          | 0/15 [00:00<?, ?it/s]

applying restraints `attach-p`:   0%|          | 0/15 [00:00<?, ?it/s]

applying restraints `pull-s`:   0%|          | 0/46 [00:00<?, ?it/s]

applying restraints `pull-p`:   0%|          | 0/46 [00:00<?, ?it/s]

applying restraints `release`:   0%|          | 0/15 [00:00<?, ?it/s]

## 03) Generate files for multiple host-guest complexes

In [9]:
host_guest_codes = {
    "acd": ["bam", "cbu"],
    "bcd": ["ben", "hex"],
}

In [10]:
for host in tqdm(host_guest_codes, desc="Host"):
    for guest in tqdm(host_guest_codes[host], desc="Guests", leave=False):
        system = BuildTaproomAPR(
            host_code=host,
            guest_code=guest,
            n_water=None,
            force_field=force_field,
            build_folder=f"{host}-{guest}/build_files",
            working_folder=f"{host}-{guest}/simulations",
            disable_progress=True,
        )
        system.build_system(n_cpus=4)

Host:   0%|          | 0/2 [00:00<?, ?it/s]

Guests:   0%|          | 0/2 [00:00<?, ?it/s]

Guests:   0%|          | 0/2 [00:00<?, ?it/s]